In [2]:
import numpy as np
import tensorflow as tf

from matplotlib import pyplot as plt
import seaborn as sns

from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

from keras.utils import pad_sequences
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical 

from transformers import BertTokenizer, TFBertModel

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import json

from sklearn.metrics import f1_score, accuracy_score
from keras.callbacks import Callback
import numpy as np
import pickle

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

In [3]:
torch.manual_seed(1)

In [4]:
train_data = json.load(open('../Dataset/BIO_Tagged/ATE_train.json'))
val_data = json.load(open('../Dataset/BIO_Tagged/ATE_val.json'))
test_data = json.load(open('../Dataset/BIO_Tagged/ATE_test.json'))

In [5]:
word2vec = pickle.load(open('Word Embeddings/word2vec.pkl', 'rb'))
glove = pickle.load(open('Word Embeddings/glove.pkl', 'rb'))
# Add another embedding here

In [6]:
def prepareData(train_data, embedding_type):
    X, Y = [], []
    for data in train_data:
        X.append(train_data[data]['text'].split())
        Y.append(train_data[data]['labels'])
        
    num_words = len(set([word.lower() for sentence in X for word in sentence]))       
    word_tokenizer = Tokenizer()                      
    word_tokenizer.fit_on_texts(X)                    
    X_encoded = word_tokenizer.texts_to_sequences(X)

    tag_tokenizer = Tokenizer()
    tag_tokenizer.fit_on_texts(Y)
    Y_encoded = tag_tokenizer.texts_to_sequences(Y)

    max_seq_length = 50

    X_padded = pad_sequences(X_encoded, maxlen=max_seq_length, padding="pre", truncating="post")
    Y_padded = pad_sequences(Y_encoded, maxlen=max_seq_length, padding="pre", truncating="post")

    X, Y = X_padded, Y_padded    

    embedding_matrix = np.zeros((num_words+1, 300))
    
    word2id = word_tokenizer.word_index
    for word, index in word2id.items():
        try:
            embedding_matrix[index, :] = embedding_type[word]
        except KeyError:
            pass    

    Y = to_categorical(Y)

    return X, Y, embedding_matrix, word_tokenizer, tag_tokenizer

In [7]:
def prepareBertData(train_data):
    X, Y = [], []
    for data in train_data:
        X.append(train_data[data]['text'].split())
        Y.append(train_data[data]['labels'])
        
    num_words = len(set([word.lower() for sentence in X for word in sentence]))    
    model = TFBertModel.from_pretrained("bert-base-uncased")
    
    word_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    word_tokenizer(X, return_tensors='tf')   
    word_tokenized_output = word_tokenizer(X, return_tensors='tf', padding=True, truncation=True)
    X_encoded = word_tokenized_output['input_ids'].numpy().tolist()

    tag_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tag_tokenizer(Y, return_tensors='tf')  
    tag_tokenized_output = tag_tokenizer(Y, return_tensors='tf', padding=True, truncation=True)
    Y_encoded = tag_tokenized_output['input_ids'].numpy().tolist()

    max_seq_length = 50

    X_padded = pad_sequences(X_encoded, maxlen=max_seq_length, padding="pre", truncating="post")
    Y_padded = pad_sequences(Y_encoded, maxlen=max_seq_length, padding="pre", truncating="post")

    X, Y = X_padded, Y_padded    

    Y = to_categorical(Y)

    return X, Y, TFBertModel, word_tokenizer, tag_tokenizer

In [8]:
data = {}
data['word2vec'] = {}
data['glove'] = {}
data['bert'] = {}

data['word2vec']['X_train'], data['word2vec']['Y_train'], data['word2vec']['train_embedding_matrix'], word_tokenizer, tag_tokenizer = prepareData(train_data, word2vec)
data['word2vec']['X_val'], data['word2vec']['Y_val'], data['word2vec']['val_embedding_matrix'], _, _ = prepareData(val_data, word2vec)
data['word2vec']['X_test'], data['word2vec']['Y_test'], data['word2vec']['test_embedding_matrix'], _, _ = prepareData(test_data, word2vec)

data['glove']['X_train'], data['glove']['Y_train'], data['glove']['train_embedding_matrix'], word_tokenizer, tag_tokenizer = prepareData(train_data, glove)
data['glove']['X_val'], data['glove']['Y_val'], data['glove']['val_embedding_matrix'],  _, _ = prepareData(val_data, glove)
data['glove']['X_test'], data['glove']['Y_test'], data['glove']['test_embedding_matrix'],  _, _ = prepareData(test_data, glove)

# data['bert']['X_train'], data['bert']['Y_train'], data['bert']['train_embedding_matrix'], word_tokenizer, tag_tokenizer = prepareBertData(train_data)
# data['bert']['X_val'], data['bert']['Y_val'], data['bert']['val_embedding_matrix'],  _, _ = prepareBertData(val_data)
# data['bert']['X_test'], data['bert']['Y_test'], data['bert']['test_embedding_matrix'],  _, _ = prepareBertData(test_data)


In [9]:
class MacroF1ScoreCallback(Callback):
    def __init__(self, train_data, val_data):
        super().__init__()
        self.train_data = train_data
        self.val_data = val_data
        self.train_f1s = []
        self.val_f1s = []

    def on_epoch_end(self, epoch, logs=None):
        train_pred = np.argmax(self.model.predict(self.train_data[0]), axis=-1)
        train_true = np.argmax(self.train_data[1], axis=-1)
        train_f1 = f1_score(train_true.flatten(), train_pred.flatten(), average='macro')

        val_pred = np.argmax(self.model.predict(self.val_data[0]), axis=-1)
        val_true = np.argmax(self.val_data[1], axis=-1)
        val_f1 = f1_score(val_true.flatten(), val_pred.flatten(), average='macro')

        self.train_f1s.append(train_f1)
        self.val_f1s.append(val_f1)
        print(f'Epoch {epoch + 1} - Train Macro-F1: {train_f1:.4f} - Val Macro-F1: {val_f1:.4f}')


In [ ]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [ ]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [ ]:
#Prepare data?
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

# Make up some training data
training_data = [(
    "the wall street journal reported today that apple corporation made money".split(),
    "B I I I O O O B I O O".split()
), (
    "georgia tech is a university in georgia".split(),
    "B I O O O O B".split()
)]

word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    print(model(precheck_sent))

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(
        300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    print(model(precheck_sent))
# We got it!

## EVALUATION

In [20]:
def get_aspect_terms(test_pred, X_test):
    aspect_terms = []
    for i in range(len(test_pred)):
        aspect_term = []
        for j in range(len(test_pred[i])):
            if test_pred[i][j] == 1:
                aspect_term.append(word_tokenizer.index_word[X_test[i][j]])
        aspect_terms.append(aspect_term)
    return aspect_terms

In [21]:
def evaluate_model(model, X_test, Y_test):
    Y_pred = np.argmax(model.predict(X_test), axis=-1)
    Y_true = np.argmax(Y_test, axis=-1)

    aspect_terms = get_aspect_terms(Y_pred, X_test)

    print(f'Macro F1 Score: {f1_score(Y_true.flatten(), Y_pred.flatten(), average="macro"):.4f}')
    print(f'Accuracy: {accuracy_score(Y_true.flatten(), Y_pred.flatten())*100:.2f}%')

    return aspect_terms

In [24]:
bilstim_crf_word2vec = load_model('Models/t2_bilstim_crf_word2vec.h5')
bilstim_crf_glove = load_model('Models/t2_bilstim_crf_glove.h5')
# bilstim_crf_bert = load_model('Models/t2_bilstim_crf_bert.h5')


print("BILSTM-CRF Word2Vec")
bilstim_crf_word2vec_aspect_terms = evaluate_model(bilstim_crf_word2vec, data['word2vec']['X_test'], data['word2vec']['Y_test'])

print()
print("BILSTM-CRF Glove")
bilstim_crf_glove_aspect_terms = evaluate_model(bilstim_crf_glove, data['glove']['X_test'], data['glove']['Y_test'])

# print()
# print("BILSTM-CRF bert")
# bilstim_crf_bert_aspect_terms = evaluate_model(bilstim_crf_bert, data['bert']['X_test'], data['bert']['Y_test'])

RNN Word2Vec
11/11 [==============================] - 0s 8ms/step
Macro F1 Score: 0.5388
Accuracy: 90.84%

RNN Glove
11/11 [==============================] - 0s 8ms/step
Macro F1 Score: 0.5311
Accuracy: 90.79%

LSTM Word2Vec
11/11 [==============================] - 1s 15ms/step
Macro F1 Score: 0.4959
Accuracy: 91.90%

LSTM Glove
11/11 [==============================] - 1s 21ms/step
Macro F1 Score: 0.5019
Accuracy: 91.61%

GRU Word2Vec
11/11 [==============================] - 1s 17ms/step
Macro F1 Score: 0.5225
Accuracy: 90.68%

GRU Glove
11/11 [==============================] - 1s 17ms/step
Macro F1 Score: 0.5280
Accuracy: 90.80%
